In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline
import plotly as px
import numpy as np
import missingno as msno # analyse des manquants 
import plotly.graph_objects as go #donut chart
import scipy.stats as scipy
from scipy.stats import pearsonr
from math import exp


In [ ]:
#upload dataset exploratoire
df=pd.read_csv("/content/drive/MyDrive/Projet4_ConsoElec/dataset.csv")


In [ ]:
dataset=df

In [ ]:
dataset.head()

In [ ]:
dataset.columns

In [ ]:
dataset=dataset[['OSEBuildingID', 'LargestPropertyUseType', 'LargestPropertyUseTypeGFA',
        'ENERGYSTARScore', 'SiteEnergyUse(kBtu)',
       'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)',
       'OtherFuelUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)',
       'log_SiteEnergyUse(kBtu)', 'log_GHGEmissions(MetricTonsCO2e)',
       'log_LargestPropertyUseTypeGFA', 'Year_slice', 'NbFloors_slice',
       'Bin_SteamUse(kBtu)',  'Bin_NaturalGas(kBtu)',
       'Bin_OtherFuelUse(kBtu)']]

In [ ]:
datacor=dataset[['LargestPropertyUseType', 'LargestPropertyUseTypeGFA',
        'ENERGYSTARScore', 
       'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)',
       'OtherFuelUse(kBtu)', 'log_SiteEnergyUse(kBtu)', 'log_GHGEmissions(MetricTonsCO2e)',
       'log_LargestPropertyUseTypeGFA', 'Year_slice', 'NbFloors_slice']]

## Corrélations des variables

In [ ]:
# Matrice des corrélations
fig, ax = plt.subplots(figsize=(15,15))  
sns.heatmap(datacor.corr(),annot = True, square=True)

NameError: ignored

In [ ]:
dataset.dropna(0, inplace=True) # comme j'uilise ce dataset pour la régression je préfère droper les données manquantes plutôt que les approximer 

In [ ]:
#Corrélation émissionsCO2/Energie
sns.set_theme(color_codes=True)

ax=sns.regplot(x='SteamUse(kBtu)', y='GHGEmissions(MetricTonsCO2e)', data=dataset)


In [ ]:
#Corrélation émissionsCO2/Energie
sns.set_theme(color_codes=True)

ax=sns.regplot(x="log_GHGEmissions(MetricTonsCO2e)", y='log_SiteEnergyUse(kBtu)', data=dataset)
#on va prédire les emissions à partir de l'énergie (r=0.89)

In [ ]:
X=dataset["log_GHGEmissions(MetricTonsCO2e)"]
Y=dataset['log_SiteEnergyUse(kBtu)']
corr, _ = pearsonr(X, Y)
print(corr)

In [ ]:
X=dataset['log_LargestPropertyUseTypeGFA']
Y=dataset['log_SiteEnergyUse(kBtu)']
corr, _ = pearsonr(X, Y)
print(corr)

In [ ]:
X=dataset['SiteEnergyUse(kBtu)']
Y=dataset["GHGEmissions(MetricTonsCO2e)"]
corr, _ = pearsonr(X, Y)
print(corr)

In [ ]:
#Corrélation 

ax=sns.regplot(y='log_LargestPropertyUseTypeGFA', x="log_GHGEmissions(MetricTonsCO2e)", data=dataset)


In [ ]:
X=dataset['log_LargestPropertyUseTypeGFA']
Y=dataset["log_GHGEmissions(MetricTonsCO2e)"]
corr, _ = pearsonr(X, Y)
print(corr)

Interprétations : Les analyses statistiques et les corrélations me permettent d'établir une liste de variables importantes dans la modélisation des consommations d'énergie et prédictions des émissions de CO2. 


Les variables non numériques sont encodées pour la modélisation
Les énergies sont binarisées. Cela abolit l'importance de l'électricité dans le model énergie car plus de 99,7% des bâtiments utilisent l'électricité.
 
Modèle émissions de CO2:Les émissions de CO2 dépendent de l'énergie consommée. Ce sont les mêmes variables que pour le modèle de prédiction de l'énergie qui sont corrélées aux émissions de CO2. Ce sont juste les intensités de corrélation qui varient. 

Remarque: L'EnergyStarScore n'est corrélée à aucune variable comme le montrait la matrice des corrélations. On evaluera son importance dans le modèle de prédiction des émissions de CO2. 
L'intensité de la relation entre dépenses énergies ou les émissions de C02 avec la variable neighboord est faible (np2=0,012, np2=0,015). je ne conserve pas cette variable pour la modélisation car son encodage est complexe. 


# PARTIE 3 : Modélisation 

In [ ]:
from sklearn import linear_model
import numpy as np
# scaling and dataset split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# OLS (moindres carrés), Ridge
from sklearn import metrics
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import RandomizedSearchCV 

#non linear
from sklearn.ensemble import RandomForestRegressor

# model evaluation
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score



In [ ]:
pip install shap # interprétation des variables dans le modèle 

In [ ]:
import shap

## Modélisation du CO2

J'ajoute l'EnergyStarScore dans mon modèle afin d'analyser sa contribution

Rappel: les variables corrélées aux émissions de CO2 sont l'énergie consommée (à prédire), l'utilisation du gaz naturel (variable binarisée),  le type de propriété et le nombre d'étages. 

### Target encoding de largestPropertyUse: target= log(Emissions de CO2)

In [ ]:
dataset.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()

In [ ]:
mean_data_=dataset[['LargestPropertyUseType', 'ENERGYSTARScore','log_LargestPropertyUseTypeGFA', 'NbFloors_slice', 'Year_slice',
        'Bin_NaturalGas(kBtu)', 'Bin_SteamUse(kBtu)', 'log_GHGEmissions(MetricTonsCO2e)']]

In [ ]:
mean_data_.info()

In [ ]:
mean_data_['enc_NbFloors']=le.fit_transform(mean_data_['NbFloors_slice'])
mean_data_['enc_Year_slice']=le.fit_transform(mean_data_['Year_slice'])

In [ ]:
mean_data_=mean_data_[['LargestPropertyUseType', 'ENERGYSTARScore',
       'log_LargestPropertyUseTypeGFA', 'enc_NbFloors', 'enc_Year_slice',
        'Bin_NaturalGas(kBtu)', 'Bin_SteamUse(kBtu)', 'log_GHGEmissions(MetricTonsCO2e)']]

Target mean encoding :

encodage par la moyenne de la cible : log_GHGEmissions

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mean_data_[['LargestPropertyUseType', 'ENERGYSTARScore', 'enc_NbFloors', 'enc_Year_slice',
        'Bin_NaturalGas(kBtu)', 'Bin_SteamUse(kBtu)', 'log_LargestPropertyUseTypeGFA', 'log_GHGEmissions(MetricTonsCO2e)']],
        mean_data_['log_GHGEmissions(MetricTonsCO2e)'], test_size=0.3, random_state=100)

In [ ]:
#Now, we will calculate the target frequency of each LargestPropertyUseType' according to the 'log_GHGEmissions(MetricTonsCO2e)' value using groupby()
target_freq=X_train.groupby(['LargestPropertyUseType'])['log_GHGEmissions(MetricTonsCO2e)'].mean()
target_freq

In [ ]:
#Converting to dictionnary
ordered_target_freq=target_freq.to_dict()
ordered_target_freq

# Les fréquences ont été calculées sur le train seulement et sont mappées sur le test 
X_train['Enc_LargestPropertyType']=X_train.LargestPropertyUseType.map(ordered_target_freq)
X_test['Enc_LargestPropertyType']=X_test.LargestPropertyUseType.map(ordered_target_freq)


In [ ]:
X_train.info()

In [ ]:
#Relation monotonique entre la variable et son encodage
fig=plt.figure()
fig=X_train.groupby(['Enc_LargestPropertyType'])['log_GHGEmissions(MetricTonsCO2e)'].mean().plot()
fig.set_title('Relation monotonique entre la variable encodée et la cible')
fig.set_ylabel('GHGEmissions(M.TonsCO2e)')

In [ ]:
X_test

In [ ]:
# je drope les colonnes pour la régression :  Largestproperty qui est maintenant encodée 
X_test.drop(['LargestPropertyUseType'],axis=1, inplace=True)
X_train.drop(['LargestPropertyUseType'],axis=1, inplace=True)

In [ ]:
datareg_=X_train[['ENERGYSTARScore',  'log_LargestPropertyUseTypeGFA', 'Enc_LargestPropertyType', 'enc_NbFloors', 'enc_Year_slice',
        'Bin_NaturalGas(kBtu)', 'Bin_SteamUse(kBtu)', 'log_GHGEmissions(MetricTonsCO2e)']]

In [ ]:
X=datareg_[datareg_.columns[:-1]]
y=datareg_['log_GHGEmissions(MetricTonsCO2e)']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=100)

In [ ]:
X_train

je ne standradise pas enregy star score car variable discrète (skew faible). Les autres sont transformées déjà par le log. 

## Modèles linéaires pour prédire les émissions de CO2

### Régression linéaire (baseline)

In [ ]:
# On crée un modèle de régression linéaire
lr = linear_model.LinearRegression()

In [ ]:
%timeit lr.fit(X_train, y_train)

In [ ]:
# évaluation du modèle en CV sur le jeu d'entrainement :
MSE_lr_baseline= cross_val_score (lr, X_train, y_train, cv=5, scoring='neg_mean_squared_error') 
r2_lr_baseline= cross_val_score (lr, X_train, y_train, cv=5, scoring='r2')

In [ ]:
#Evaluation du modèle de régression linéaire (baseline)

# sur le jeu train en cross validation :
print ('MSE_lr(CV):', np.mean(MSE_lr_baseline*(-1)))
print ('r2_lr (CV):', np.mean(r2_lr_baseline))

In [ ]:
#j'applique le modèle 
y_pred_lr=lr.predict(X_test)

In [ ]:
plt.figure(figsize = ( 6 , 6 )) 
  
sns.regplot(x=y_test, y=y_pred_lr, color='coral') #true vs predict
  
plt.xlabel( "y_test (log_kBtu)" , size = 12 ) 
  
plt.ylabel(  "y_pred (log_kBtu)"  , size = 12 ) 
  
plt.title( "True vs Predicts" , size = 15 ) 
  
plt.show() 


In [ ]:
# regression intercept
print('Intercept:' ,lr.intercept_) 

#score de la régression sur le test
print('R2_test:', lr.score(X_test, y_test))
print ('MSE (lr, pred):', mean_squared_error (y_test, y_pred_lr))


In [ ]:
df_coef_lr= pd.DataFrame(lr.coef_, index=X_train.columns)
df_coef_lr.columns=['lr Baseline']
df_coef_lr

In [ ]:
df_coef_lr.plot.bar(title='Coefficients')

In [ ]:
explainer = shap.LinearExplainer(lr, X_train)
shap_values = explainer.shap_values(X_train)
f = plt.figure()
shap.summary_plot(shap_values, X_train)

In [ ]:
df_y_predLR= pd.DataFrame(y_pred_lr)

In [ ]:
df_y_predLR= pd.DataFrame(y_pred_lr)

# plot for residual error
 
## setting plot style
plt.style.use('fivethirtyeight')
 

## plotting residual errors in test data
plt.scatter(x=df_y_predLR.index, y=lr.predict(X_test) - y_test,
            color = "blue", s = 10, label = 'Test data')
#plt.scatter((X_train).index, lr.predict(X_train) - y_train,
           # color = "coral", s = 10, label = 'Train data')
 
## plotting line for zero residual error
plt.hlines(y = 0, xmin = 0, xmax = 250, linewidth = 2)
 
## plotting legend
plt.legend(loc = 'upper right')
 
## plot title
plt.ylabel('Error Rate')
plt.title("Residual errors")

 
## function to show plot
plt.show()


### Regression Ridge CV

In [ ]:
n_alphas = 200
alphas=np.logspace(-3, 3, n_alphas) 

#Entraînement du modèle 
from sklearn.linear_model import Ridge
ridge = linear_model.Ridge()

coefs = []
squared_r = []
errors_MSE = []


for a in alphas:
    ridge.set_params(alpha=a) 
    ridge.fit(X_train, y_train)
    coefs.append(ridge.coef_)
    score_MSE_R=cross_val_score (ridge, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    score_r2_R=cross_val_score (ridge, X_train, y_train, cv=5, scoring='r2')
    squared_r.append([np.mean(r2_lr_baseline), np.mean(score_r2_R)])
    errors_MSE.append([np.mean(MSE_lr_baseline*(-1)), -np.mean(score_MSE_R)])
  

In [ ]:
errors_MSE

In [ ]:
# Fonction qui recherche l'erreur minimale et donne son index
def min_alpha(errors):
  index=0
  min_val=10
  for i, error in enumerate (errors): #fonction qui à chaque boucle écrase min val et index
    if error[1]<min_val:
      min_val=error[1]
      index=i
  return index #on stocke l'index de l'erreur minimale et à chaque fois qu'on trouve une erreur plus petite on remplace l'index et le min val de maniere à mettre à jour la valeur la plus petite. 
 

In [ ]:
min(errors_MSE)

In [ ]:
#Chemin de régularisation 
ax = plt.gca()

ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('poids')
plt.title( 'Chemin de regularisation')
plt.axis('tight')
plt.show()

In [ ]:
# Evolution de MSE en fonction de alpha (Train)
ax = plt.gca()

ax.plot(alphas, errors_MSE)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('MSE')
plt.title( 'Erreur en fonction de alpha')
plt.axis('tight')
plt.show()

In [ ]:
# Evolution de R2 en fonction du coeff alpha 
ax = plt.gca()

ax.plot(alphas, squared_r)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('R2')
plt.title("R2 en fonction de alpha")
plt.axis('tight')
plt.show()

In [ ]:
index=min_alpha(errors_MSE)
index

In [ ]:
alphas[index]

In [ ]:
coefs[index]

In [ ]:
errors_MSE[index]

In [ ]:
squared_r[index]

In [ ]:
#J'applique le meilleur alpha sur la regression Ridge: 
ridge_b = linear_model.Ridge(alpha=alphas[index])
%timeit ridge_b.fit(X_train, y_train)

In [ ]:
ridge_b.score(X_train, y_train)

In [ ]:
y_pred_r=ridge_b.predict(X_test)

In [ ]:
plt.figure(figsize = ( 6 , 6 )) 
  
sns.regplot(x=y_test, y=y_pred_r, color='coral') #true vs predict
  
plt.xlabel( "y_test (log_kBtu)" , size = 12 ) 
  
plt.ylabel(  "y_pred (log_kBtu)"  , size = 12 ) 
  
plt.title( "True vs Predicts" , size = 15 ) 
  
plt.show() 


In [ ]:
# regression intercept
print('Intercept:' ,ridge_b.intercept_) 

#score de la régression sur le test
print('R2_test:', ridge_b.score(X_test, y_test))
print ('MSE (lr, pred):', mean_squared_error (y_test, y_pred_r))

In [ ]:
df_y_pred_r= pd.DataFrame(y_pred_r)

# plot for residual error
 
## setting plot style
plt.style.use('fivethirtyeight')
 

## plotting residual errors in test data
plt.scatter(x=df_y_pred_r.index, y=ridge_b.predict(X_test) - y_test,
            color = "blue", s = 10, label = 'Test data')
#plt.scatter((X_train).index, lr.predict(X_train) - y_train,
           # color = "coral", s = 10, label = 'Train data')
 
## plotting line for zero residual error
plt.hlines(y = 0, xmin = 0, xmax = 250, linewidth = 2)
 
## plotting legend
plt.legend(loc = 'upper right')
 
## plot title
plt.ylabel('Error Rate')
plt.title("Residual errors")

 
## function to show plot
plt.show()

In [ ]:
df_coef_R= pd.DataFrame(ridge_b.coef_, index=X_train.columns)
df_coef_R.columns=['Ridge']
df_coef_R

In [ ]:
df_coef_R.index

In [ ]:
df_coef_R.plot.bar(title='Coefficients')

In [ ]:
explainer = shap.LinearExplainer(ridge_b, X_train)
shap_values = explainer.shap_values(X_train)
f = plt.figure()
shap.summary_plot(shap_values, X_train)

In [ ]:
explainer = shap.LinearExplainer(ridge_b, X_test)
shap_values = explainer.shap_values(X_test)
f = plt.figure()
shap.summary_plot(shap_values, X_test)

In [ ]:
#Evaluation de la régression Ridge:
# sur le jeu train en cross validation :
print ('MSE_BaselineCV vs RidgeCV:', errors_MSE[index])
print ('r2__BaselineCV vs RidgeCV:', squared_r[index])

#Evaluation du modèle sur le test :
print ('R2_test:', r2_score (y_test, y_pred_r))
print ('MSE_test:', mean_squared_error (y_test, y_pred_r))


Regression Ridge avec alpha optimal en CV permet de diminuer la MSE et le modèle est régularisé. C'est le meilleur modèle linéaire. Le lasso régularise très peu et la MSE ne diminue pas. 

### Lasso

In [ ]:
#Entraînement du lasso sur le jeu d'entraînement 
n_alphas = 200
a = np.logspace(-3, 3, n_alphas)
lasso = linear_model.Lasso()

coefs_l = []
errors_l = []
squared_r_l=[]

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X_train, y_train) 
    coefs_l.append(lasso.coef_)
    score_MSE_l =cross_val_score (lasso, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    score_r2_l=cross_val_score (lasso, X_train, y_train, cv=5, scoring='r2')
    squared_r_l.append([np.mean(r2_lr_baseline), np.mean(score_r2_l)])
    errors_l.append([np.mean(MSE_lr_baseline*(-1)), -np.mean(score_MSE_l)])
    

In [ ]:
#Chemin de régularisation (lasso)
ax = plt.gca()

ax.plot(alphas, coefs_l)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('poids')
plt.title('Lasso coefficients, Chemin de regularisation')
plt.axis('tight')
plt.show()

In [ ]:

ax = plt.gca()

ax.plot(alphas, errors_l)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('error')
plt.axis('tight')
plt.title('Erreur en fonction de alpha')
plt.show()

In [ ]:
ax = plt.gca()

ax.plot(alphas, squared_r_l)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('R2')
plt.title("R2 en fonction de alpha")
plt.axis('tight')
plt.show()

In [ ]:
min(errors_l)

In [ ]:
#alpha optimal 
index=min_alpha(errors_l)


In [ ]:
index

In [ ]:
errors_l[index] 

In [ ]:
alphas[index]

In [ ]:
#j'applique le meilleur alpha sur le modèle lasso:

lasso = linear_model.Lasso(alpha=alphas[index])
%timeit lasso.fit(X_train, y_train)


In [ ]:
y_pred_l=lasso.predict(X_test)

In [ ]:
coefs_l[index]

In [ ]:
lasso.coef_

In [ ]:
df_coef_l= pd.DataFrame(lasso.coef_, index=X_train.columns) 
df_coef_l.columns=['Lasso']
df_coef_l

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
explainer = shap.LinearExplainer(lasso, X_train)
shap_values = explainer.shap_values(X_train)
f = plt.figure()
shap.summary_plot(shap_values, X_train)

In [ ]:
#Evaluation du modèle LassoS:
# sur le jeu train en cross validation :
print ('MSE_baselineCV vs LassoCV:',errors_l[index] )
print ('r2_baselineCV vs LassoCV:', squared_r_l[index])

#Evaluation du modèle sur le test :
print ('r2 (test):', r2_score (y_test, y_pred_l))
print ('MSE (test):', mean_squared_error (y_test, y_pred_l))

### Conclusion des modèles linéaires
La régularisation Ridge permet de légèrement corriger le modèle. C'est le meilleur modèle linéaire. La MSE test est meilleure que la MSE train et l'erreur est stable au fur et à mesure des prédictions.

 L'EnergyStar score a un coefficient très faible dans les modèles linéaires.  Les coefficients les plus forts sont l'utilisation de la vapeur d'eau et du gaz puis la surface des bâtiments.
L'analyse par shap values montre que l'Energy Star score a une contribution modérée dans le modèle, après la surface et l'utilisation du gaz. 

Je vais tester un modèle non linéaire pour voir si il est plus adapté.

##Random Forest Regressor (RandomizedSearchCV)

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)] # je test 10 estimateurs entre 200 et 1000
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 50, num = 10)] #nb étages dans l'arbre 
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10] #nb échantillons par feuille et créer un noeud 
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4] #nb min d'échantillon par feuille : permet de limiter la profondeur de l'arbre et donc l'overfitting 
# Method of selecting samples for training each tree
bootstrap = [True, False] # randomnisation et aggrégation des résultats : permet d'éviter l'overfitting

In [ ]:
# Create the random grid

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
              

In [ ]:
# Utilisation de la random_grid pour trouver les meilleurs paramètres du modèles
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 5 fold cross validation, 
# search across 50 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, cv = 5, verbose=2, random_state=30, n_jobs = -1, scoring=['neg_mean_squared_error','r2'], refit='neg_mean_squared_error') #refit: cherhce à optimiser la mse # par défaut le score est le R2 mais comme le modèle n'est pa slinéaire il faut changer 
# Fit the random search model
rf_random.fit(X_train, y_train) 

In [ ]:
rf_random.best_params_

In [ ]:
n_estimators = rf_random.best_params_['n_estimators']
min_samples_split=  rf_random.best_params_['min_samples_split']
min_samples_leaf=rf_random.best_params_['min_samples_leaf']
max_features=rf_random.best_params_['max_features']
max_depth =rf_random.best_params_['max_depth']
bootstrap =rf_random.best_params_['bootstrap']

In [ ]:
# Applications des best params du randomizedSearchCV

rf_best = RandomForestRegressor(n_estimators = n_estimators,
min_samples_split=min_samples_split ,
min_samples_leaf=min_samples_leaf,
max_features=max_features,
max_depth =max_depth,
bootstrap =bootstrap)

# Fit the random search model
%timeit rf_best.fit(X_train, y_train)

In [ ]:
# Evaluation de la RF: MSE 
#c'est la neg mse en validation croisée 
# 5 fold, 5 scores par random forest et celle qui a le meilleur score 
rf_random.best_score_
MSE_RF_best=(rf_random.best_score_)*(-1)

In [ ]:
MSE_RF_best

In [ ]:
df_Features_importance_RF_Best= pd.DataFrame(rf_best.feature_importances_, index=X_train.columns)
df_Features_importance_RF_Best.columns=['Random Forest (best)']
df_Features_importance_RF_Best

In [ ]:
df_Features_importance_RF_Best.plot.bar(title='Feature importance')

In [ ]:
explainer = shap.TreeExplainer(rf_best, X_train)
shap_values = explainer.shap_values(X_train)
f = plt.figure()
shap.summary_plot(shap_values, X_train)

In [ ]:
explainer = shap.TreeExplainer(rf_best, X_test)
shap_values = explainer.shap_values(X_test)
f = plt.figure()
shap.summary_plot(shap_values, X_test)

La RF n'apporte pas une meilleure erreur et le temps d'entrainement est plus long. Le % de contribution de l'EnergyStar score dans le modèle RF est plus important et légèrement plus modéré dans l'analyse shap. 
Je ne choisis pas ce modèle.

## Choix du modèle : RidgeCV

In [ ]:
from math import exp

In [ ]:
#Evaluation de la régression Ridge:
# sur le jeu train en cross validation :
print ('MSE_BaselineCV vs RidgeCV:', errors_MSE[index])
print ('r2__BaselineCV vs RidgeCV:', squared_r[index])
print('MSE RF_Best:', MSE_RF_best)

#Evaluation du modèle sur le test :
print ('R2_test Ridge:', r2_score (y_test, y_pred_r))
print ('MSE_test Ridge:', mean_squared_error (y_test, y_pred_r))

# Repasser les MSE en KbTu

print("MSE(Metric/tons/C02) Ridge:", exp(0.487630140151284))
print("MSE(Metric/tons/C02) lr_baseline:", exp(0.487630165218888))

In [ ]:
Features_results_CO2= pd.concat([df_coef_lr,df_coef_R, df_coef_l,df_Features_importance_RF_Best], axis=1)
Features_results_CO2

Conclusion : Le meilleur modèle est la régression linéaire régularisée par Ridge. 


## Select From Model sur Ridge
Importance des variables dans le meilleur modèle 

In [ ]:
#SElect From Model: voir importance des variables en particulier l'EnergyStarscore 
from sklearn.feature_selection import SelectFromModel

In [ ]:
selector = SelectFromModel(estimator=ridge_b).fit(X_train, y_train) # threshold=mean(coef)

In [ ]:
selector.threshold_


In [ ]:
selector.get_support()

In [ ]:
X_train_Select= selector.transform(X_train)

In [ ]:
# Pour récupérer les variables selectionnées ou non (array True ou Flase)
list_columns=X_train.columns
new_columns=[]
for i, columns in enumerate (list_columns):
  if selector.get_support()[i]== True :
    new_columns.append(columns)

In [ ]:
new_columns

In [ ]:
df_X_train_Select = pd.DataFrame(X_train_Select, columns=new_columns)

In [ ]:
df_X_train_Select

## Conclusions

L'erreur la plus faible est dans le modèle linéaire Ridge(CV). L'erreur ramenée en MetricTonsCO2 n'est pas négligeable.
Je choisis la régularisation Ridge, modèle régularisé, plus stable que le lasso, meilleure MSE.
La surface, le type de bâtiments et l'utilisation du gaz naturel sont les variables qui ont le plus d'importance dans modèle choisi pour prédire le CO2.
L'EnergyStarScore est de poids modéré dans le modèle (interprétation shap values), son poids est inférieur au poids moyen des variables, c'est pourquoi elle n'est pas retenue par le SElectFromModel.
Le temps d'entrainement de Ridge (alpha optimal), une fois les meilleurs paramètres obtenus, est comparable au modèle linéaire et bien inférieur à la RF.

Testes supplémentaires : Ridge sans EnergySatrScore

In [ ]:
datareg_=datareg_[['log_LargestPropertyUseTypeGFA', 'Enc_LargestPropertyType', 'enc_NbFloors', 'enc_Year_slice',
        'Bin_NaturalGas(kBtu)', 'Bin_SteamUse(kBtu)', 'log_GHGEmissions(MetricTonsCO2e)']]

In [ ]:
X=datareg_[datareg_.columns[:-1]]
y=datareg_['log_GHGEmissions(MetricTonsCO2e)']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=100)

In [ ]:
lr = linear_model.LinearRegression()

In [ ]:
%timeit lr.fit(X_train, y_train)

In [ ]:
# évaluation du modèle en CV sur le jeu d'entrainement :
MSE_lr_baseline= cross_val_score (lr, X_train, y_train, cv=5, scoring='neg_mean_squared_error') 
r2_lr_baseline= cross_val_score (lr, X_train, y_train, cv=5, scoring='r2')

In [ ]:
MSE_lr_baseline

In [ ]:
r2_lr_baseline

In [ ]:
n_alphas = 200
alphas=np.logspace(-3, 3, n_alphas) 

#Entraînement du modèle 
from sklearn.linear_model import Ridge
ridge = linear_model.Ridge()

coefs = []
squared_r = []
errors_MSE = []


for a in alphas:
    ridge.set_params(alpha=a) 
    ridge.fit(X_train, y_train)
    coefs.append(ridge.coef_)
    score_MSE_R=cross_val_score (ridge, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    score_r2_R=cross_val_score (ridge, X_train, y_train, cv=5, scoring='r2')
    squared_r.append([np.mean(r2_lr_baseline), np.mean(score_r2_R)])
    errors_MSE.append([np.mean(MSE_lr_baseline*(-1)), -np.mean(score_MSE_R)])
  

In [ ]:
min(errors_MSE)